In [ ]:
import pandas as pd
from pyprojroot import here
import matplotlib.pyplot as plt
import numpy as np


In [ ]:
data = pd.read_csv(here() / 'data/base/data.csv')

In [ ]:
categories = ['Not at all', 'Slight', 'Moderate', 'Severe', 'Very severe']
question = "In your ED, how serious threat to patient safety do you consider crowding to be overall?"

In [ ]:
a = data.groupby(['Country', 'Hospital Name'])[question].value_counts(normalize=True).unstack().fillna(0)

In [ ]:
unique_hospitals = a.reset_index().groupby('Country')['Hospital Name'].nunique()
unique_hospitals.name = 'Country'

In [ ]:
matrix = a.groupby('Country').sum().div(unique_hospitals, axis=0)

In [ ]:
all_responses = ((matrix.sum() / matrix.sum().sum()) * 100)

In [ ]:
relative = matrix.div(matrix.sum(axis=1), axis=0)
relative = (relative*100).round(0)
relative = relative[categories]
all_responses = all_responses[categories]

In [ ]:
relative = relative.T

In [ ]:
fig, axs = plt.subplots(
    ncols=2, 
    figsize=(10,3), 
    gridspec_kw={
        'width_ratios':[10,1]
        },
    sharey=True
    )

axs[0].imshow(relative, cmap='Greys', vmin=0, vmax=100, aspect='auto')

axs[0].set_yticks(np.arange(len(relative.index)))
axs[0].set_yticklabels(relative.index)

axs[0].set_xticks(np.arange(len(relative.columns)))
axs[0].set_xticklabels(relative.columns)
plt.setp(axs[0].get_xticklabels(), rotation=45, ha='right', rotation_mode='anchor')


all_responses.plot.barh(ax=axs[1], color='lightgrey', width=.8)
axs[1].set_xlim([0,100])

# Annotate each cell with percentage
arr = relative.values 
for i in range(arr.shape[0]):
    for j in range(arr.shape[1]):
        if arr[i,j]<80:
            color = 'black'
        else:
            color='white'
        axs[0].text(j, i, f"{arr[i, j]:.0f}", ha="center", va="center", color=color)

plt.tight_layout()

plt.savefig(here() / 'output/plots/severity.png', dpi=300, bbox_inches='tight')